In [32]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [33]:
from crewai import Task,Agent,Crew,LLM
from dotenv import  load_dotenv
import os
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

os.environ["OPENAI_API_KEY"] = "dummy"

llm = LLM(model="gemini/gemini-2.5-flash",
          provider="gemini",
          temperature=0.7,
          api_key=api_key)

from crewai.memory import LongTermMemory
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage

# Configure custom storage location
custom_storage_path = "./storage"
os.makedirs(custom_storage_path, exist_ok=True)


In [34]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=llm,
)

In [35]:
support_agent

Agent(role=Senior Support Representative, goal=Be the most friendly and helpful support representative in your team, backstory=You work at crewAI (https://crewai.com) and  are now working on providing support to {customer}, a super important customer  for your company.You need to make sure that you provide the best support!Make sure to provide full complete answers,  and make no assumptions.)

In [36]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	llm=llm,
)

In [37]:
support_quality_assurance_agent

Agent(role=Support Quality Assurance Specialist, goal=Get recognition for providing the best support quality assurance in your team, backstory=You work at crewAI (https://crewai.com) and are now working with your team on a request from {customer} ensuring that the support representative is providing the best support possible.
You need to make sure that the support representative is providing fullcomplete answers, and make no assumptions.)

In [38]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [39]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [40]:
docs_scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Tool Name: Read website content\nTool Arguments: {}\nTool Description: A tool that can be used to read https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/'s content.", env_vars=[], args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.FixedScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x104231870>, result_as_answer=False, max_usage_count=None, current_usage_count=0, website_url='https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/', cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 'keep-a

In [41]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [42]:
inquiry_resolution

Task(description={customer} just reached out with a super important ask:
{inquiry}

{person} from {customer} is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry., expected_output=A detailed, informative response to the customer's inquiry that addresses all aspects of their question.
The response should include references to everything you used to find the answer, including external data or solutions. Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout.)

In [43]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [44]:
quality_assurance_review

Task(description=Review the response drafted by the Senior Support Representative for {customer}'s inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered., expected_output=A final, detailed, and informative response ready to be sent to the customer.
This response should fully address the customer's inquiry, incorporating all relevant feedback and improvements.
Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout.)

In [45]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  manager_llm=llm,
  memory=True,
  long_term_memory=LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path=f"{custom_storage_path}/memory.db"
        )
    )
)

In [46]:
crew

Crew(id=36b21d1d-048e-4fc7-8b0f-c55faefbca58, process=sequential, number_of_agents=2, number_of_tasks=2)

In [47]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 36b21d1d-048e-4fc7-8b0f-c55faefbca58                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-09-24 11:25:05,984 - 8430624960 - rag_storage.py-rag_storage:125 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-09-24 11:25:06,357 - 8430624960 - rag_storage.py-rag_storage:125 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - More context and information about "Crew" and "memory" should be provided to answer the question             │
│  accurately.                                                                                                    │
│  - Directly ask clarifying questions to the user to avoid ambiguity.                                            │
│  - More direct clarification questions to ensure understanding of the user's request                            │
│  - Provide initial examples of potential solutions, even if further clarification is needed                     │
│  - Include a more comprehensive list of ways to add memory to a Crew based on different interpretations of      │
│  'memory'                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 805.83ms ────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: Okay, DeepLearningAI needs help with adding memory to their Crew. That's great, I can definitely      │
│  help Andrew Ng with that! To provide the best possible support, I need to figure out exactly what kind of      │
│  "memory" he's referring to. There are a few possibilities:                                                     │
│  1.  **Task Execution Memory:** Does he want the Crew to remember what tasks have already been executed? This   │
│  would involve storing task execution history.                                                                  │
│  2.  **Agent Memory:** Does he want the agents within the Crew to have individual memories, allowing them to    │
│  recall past interactions or information they've processed?                                                     │
│  3.  **Shared Crew Knowledge:** Does he want the Crew to have a shared knowledge base that all agents can       │
│  access and contribute to?                                                                                      │
│  To cover my bases, I will use the `Read website content` tool, to find information about "Crew" and how to     │
│  set it up, specifically looking for information about "memory".                                                │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure    │
│  that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer    │
│  support.                                                                                                       │
│  Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly    │
│  tone.                                                                                                          │
│  Check for references and sources used to  find the information, ensuring the response is well-supported and    │
│  leaves no questions unanswered.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hi Andrew,                                                                                                     │
│                                                                                                                 │
│  Thanks for reaching out! I'm happy to help you with setting up memory for your Crew. To best assist you, I     │
│  need a little more information about what you mean by "memory" in this context. To help me understand your     │
│  needs better, please consider the following:                                                                   │
│                                                                                                                 │
│  **Please tell me more about what you're trying to achieve:**                                                   │
│                                                                                                                 │
│  *   **Specific Use Case:** Can you describe the specific problem you're trying to solve with memory in your    │
│  Crew?                                                                                                          │
│  *   **Data Type:** What type of information do you need your Crew or agents to remember (e.g., facts,          │
│  previous interactions, code snippets)?                                                                         │
│  *   **Persistence:** Do you need the memory to persist across multiple sessions, or is it sufficient for it    │
│  to last only during a single Crew execution?                                                                   │
│                                                                                                                 │
│  In the meantime, here are some potential approaches, depending on what you're trying to achieve:               │
│                                                                                                                 │
│  **1. Task Execution Memory:**                                                                                  │
│                                                                                                                 │
│  *   **Description:** Storing task execution history to avoid redundant tasks or track progress.                │
│  *   **Approach:** CrewAI doesn't natively store task execution history. You would need to implement a custom   │
│  solution using a database or logging system.                                                                   │
│  *   **Example:** You could create a simple log file or database table to record the task name, input, output,  │
│  and timestamp for each executed task.                                                                          │
│                                                                                                                 │
│  **2. Agent Memory:**                                                                                           │
│                                                                                                                 │
│  *   **Description:** Allowing agents to recall past interactions or processed information. This is the most    │
│  common type of memory requested.                                                                               │
│  *   **Approach:** Using tools with memory capabilities

2025-09-24 11:25:30,137 - 8430624960 - rag_storage.py-rag_storage:94 - ERROR: Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-09-24 11:25:33,411 - 8430624960 - rag_storage.py-rag_storage:94 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-09-24 11:25:33,719 - 8430624960 - rag_storage.py-rag_storage:94 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-09-24 11:25:34,027 - 8430624960 - rag_storage.py-rag_storage:94 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-09-24 11:25:34,319 - 8430624960 - rag_storage.py-rag_storage:94 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-09-24 11

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 805ecaa5-1005-451d-a06c-6384aa02b5fa                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 36b21d1d-048e-4fc7-8b0f-c55faefbca58                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hi Andrew,                                                                                       │
│                                                                                                                 │
│  Thanks for reaching out! I'm happy to help you with setting up memory for your Crew. To best assist you, I     │
│  need a little more information about what you mean by "memory" in this context. To help me understand your     │
│  needs better, please consider the following:                                                                   │
│                                                                                                                 │
│  **Please tell me more about what you're trying to achieve:**                                                   │
│                                                                                                                 │
│  *   **Specific Use Case:** Can you describe the specific problem you're trying to solve with memory in your    │
│  Crew?                                                                                                          │
│  *   **Data Type:** What type of information do you need your Crew or agents to remember (e.g., facts,          │
│  previous interactions, code snippets)?                                                                         │
│  *   **Persistence:** Do you need the memory to persist across multiple sessions, or is it sufficient for it    │
│  to last only during a single Crew execution?                                                                   │
│                                                                                                                 │
│  In the meantime, here are some potential approaches, depending on what you're trying to achieve:               │
│                                                                                                                 │
│  **1. Task Execution Memory:**                                                                                  │
│                                                                                                                 │
│  *   **Description:** Storing task execution history to avoid redundant tasks or track progress.                │
│  *   **Approach:** CrewAI doesn't natively store task execution history. You would need to implement a custom   │
│  solution using a database or logging system.                                                                   │
│  *   **Example:** You could create a simple log file or database table to record the task name, input, output,  │
│  and timestamp for each executed task.                                                                          │
│                                                                                                                 │
│  **2. Agent Memory:**                                                                                           │
│                                                                                                                 │
│  *   **Description:** Allowing agents to recall past interactions or processed information. This is the most    │
│  common type of memory requested.                     

In [48]:
from IPython.display import Markdown
Markdown(result.raw)

Hi Andrew,

Thanks for reaching out! I'm happy to help you with setting up memory for your Crew. To best assist you, I need a little more information about what you mean by "memory" in this context. To help me understand your needs better, please consider the following:

**Please tell me more about what you're trying to achieve:**

*   **Specific Use Case:** Can you describe the specific problem you're trying to solve with memory in your Crew?
*   **Data Type:** What type of information do you need your Crew or agents to remember (e.g., facts, previous interactions, code snippets)?
*   **Persistence:** Do you need the memory to persist across multiple sessions, or is it sufficient for it to last only during a single Crew execution?

In the meantime, here are some potential approaches, depending on what you're trying to achieve:

**1. Task Execution Memory:**

*   **Description:** Storing task execution history to avoid redundant tasks or track progress.
*   **Approach:** CrewAI doesn't natively store task execution history. You would need to implement a custom solution using a database or logging system.
*   **Example:** You could create a simple log file or database table to record the task name, input, output, and timestamp for each executed task.

**2. Agent Memory:**

*   **Description:** Allowing agents to recall past interactions or processed information. This is the most common type of memory requested.
*   **Approach:** Using tools with memory capabilities, such as vector databases or knowledge graphs.
*   **Example using ChromaDB:**

    ```python
    from crewai import Agent
    from langchain.vectorstores import ChromaDB
    from langchain.embeddings import OpenAIEmbeddings
    from langchain.chains import RetrievalQA
    from langchain.llms import OpenAI
    
    # Initialize ChromaDB
    embedding = OpenAIEmbeddings()
    vectordb = ChromaDB(persist_directory="chroma_db", embedding_function=embedding)
    
    # Load data into ChromaDB (example)
    # vectordb.add_texts(["CrewAI is awesome!", "Langchain is powerful."])
    
    # Create retriever
    retriever = vectordb.as_retriever()
    
    # Create QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    
    class MemoryTool():
        def use(self, query):
            return qa_chain({"query": query})['result']
    
    # Create Agent with Memory Tool
    memory_tool = MemoryTool()
    agent = Agent(
        role='Researcher',
        goal='Use memory to provide accurate information.',
        backstory='A researcher with access to a knowledge base.',
        tools=[memory_tool],
        verbose=True
    )
    
    # Example usage: First add texts, then query it.
    # vectordb.add_texts(["CrewAI was created by Joao Moura."])
    # print(agent.run("Who created CrewAI?"))
    ```

*   **Explanation:** This example demonstrates how to integrate ChromaDB with an agent. The agent can use the `MemoryTool` to query the vector database and retrieve relevant information.  Remember to install the necessary packages: `pip install chromadb langchain openai`.

**3. Shared Crew Knowledge:**

*   **Description:** A shared knowledge base accessible to all agents in the Crew.
*   **Approach:** Similar to agent memory, but the knowledge base is shared among all agents.
*   **Example:** Using a shared ChromaDB instance or a knowledge graph that all agents can access.  The setup would be similar to the Agent Memory example, but the `vectordb` instance would be accessible by all agents in the crew.

**General Approaches & Considerations:**

*   **Using Tools for Memory:** Equip agents with tools that allow them to access and store information. Vector databases (ChromaDB, Pinecone, Weaviate) are excellent choices.
*   **Chaining Tasks:** The output of one task can be used as input for the next.
*   **Custom Agent Logic:** Implement custom logic within agents to manage their own memory (e.g., storing information in local variables or external data stores).

**Security Considerations for Persistent Storage:**

*   **Data Encryption:** Encrypt sensitive data before storing it in a persistent storage solution.
*   **Access Control:** Implement strict access control policies to limit who can access the stored data.
*   **Secure Storage:** Choose a secure storage solution with appropriate security measures.
*   **Regular Audits:** Conduct regular security audits to identify and address potential vulnerabilities.

**FAQ:**

*   **Q: How do I choose the right memory solution for my Crew?**
    *   **A:** Consider the type of information you need to store, the frequency of access, the required persistence, and the security requirements.
*   **Q: Can I use multiple memory solutions in a single Crew?**
    *   **A:** Yes, you can combine different approaches to achieve the desired memory behavior.
*   **Q: What are the performance implications of using memory?**
    *   **A:** Memory access can introduce overhead. Optimize your memory usage to minimize performance impact.

**Troubleshooting Tips:**

*   **Ensure the database is running:** If you're using a database like ChromaDB, make sure it's running before you start your Crew.
*   **Check API keys:** If you're using an external service, make sure your API keys are correctly configured.
*   **Verify data formats:** Ensure that the data you're storing and retrieving is in the correct format.

**Relevant Documentation:**

*   [CrewAI Documentation](https://www.crewai.com/docs/): Comprehensive documentation for CrewAI.
*   [Langchain Documentation](https://python.langchain.com/docs/): Documentation for Langchain, which CrewAI uses.
*   [ChromaDB Documentation](https://www.trychroma.com/docs): Documentation for ChromaDB.

Could you please clarify which type of "memory" you're interested in, along with the details requested at the beginning of this response? Once I have a better understanding of your needs, I can provide more specific guidance and code examples.

I look forward to hearing from you and helping you build an amazing Crew!

Best regards,

Your Senior Support Representative